## Deliverable 2. Create a Customer Travel Destinations Map.

In [1]:
# Dependencies and Setup
import pandas as pd
import requests
import gmaps

# Import API key
from config import g_key

# Configure gmaps API key
gmaps.configure(api_key=g_key)

In [2]:
# 1. Import the WeatherPy_database.csv file. 
city_data_df = pd.read_csv("../Weather_Database/WeatherPy_database.csv")
city_data_df.head()

,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
0,0,Punta Arenas,CL,-53.1500,-70.9167,39.31,65,0,12.66,clear sky
1,1,Atuona,PF,-9.8000,-139.0333,78.06,74,13,13.40,few clouds
2,2,Busselton,AU,-33.6500,115.3333,56.08,53,64,8.39,broken clouds
3,3,Karratha,AU,-20.7377,116.8463,87.64,18,0,5.88,clear sky
4,4,Hobart,AU,-42.8794,147.3294,61.27,64,75,1.01,light rain


In [3]:
# 2. Prompt the user to enter minimum and maximum temperature criteria 
min_temp = float(input("What is the minimum temperature you would like for your trip? "))
max_temp = float(input("What is the maximum temperature you would like for your trip? "))

What is the minimum temperature you would like for your trip? 70
What is the maximum temperature you would like for your trip? 90


In [4]:
# 3. Filter the city_data_df DataFrame using the input statements to create a new DataFrame using the loc method.
preferred_cities_df = city_data_df.loc[(city_data_df["Max Temp"] <= max_temp) & \
                                       (city_data_df["Max Temp"] >= min_temp)]
preferred_cities_df.head(10)

,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
1,1,Atuona,PF,-9.8000,-139.0333,78.06,74,13,13.40,few clouds
3,3,Karratha,AU,-20.7377,116.8463,87.64,18,0,5.88,clear sky
7,7,Kaveripattinam,IN,12.4207,78.2166,74.93,87,100,5.64,overcast clouds
10,10,Bilma,NE,18.6853,12.9164,77.67,21,69,0.89,broken clouds
17,17,Opotiki,NZ,-38.0833,177.2833,71.38,50,13,7.78,few clouds
18,18,Airai,TL,-8.9266,125.4092,80.69,28,9,5.97,clear sky
19,19,Dhidhdhoo,MV,6.8833,73.1000,83.14,78,23,17.67,few clouds
21,21,Bengkulu,ID,-3.8004,102.2655,82.58,71,15,7.76,few clouds
22,22,Vanimo,PG,-2.6741,141.3028,85.77,72,80,7.56,broken clouds
23,23,Bousso,TD,10.4795,16.7117,77.52,77,100,4.27,overcast clouds


In [5]:
# 4a. Determine if there are any empty rows.
preferred_cities_df.isna()

,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
1,False,False,False,False,False,False,False,False,False,False
3,False,False,False,False,False,False,False,False,False,False
7,False,False,False,False,False,False,False,False,False,False
10,False,False,False,False,False,False,False,False,False,False
17,False,False,False,False,False,False,False,False,False,False
...,...,...,...,...,...,...,...,...,...,...
681,False,False,False,False,False,False,False,False,False,False
682,False,False,False,False,False,False,False,False,False,False
683,False,False,False,False,False,False,False,False,False,False
688,False,False,False,False,False,False,False,False,False,False


In [6]:
# 4b. Drop any empty rows and create a new DataFrame that doesn’t have empty rows.
clean_df = preferred_cities_df.dropna()
clean_df

,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
1,1,Atuona,PF,-9.8000,-139.0333,78.06,74,13,13.40,few clouds
3,3,Karratha,AU,-20.7377,116.8463,87.64,18,0,5.88,clear sky
7,7,Kaveripattinam,IN,12.4207,78.2166,74.93,87,100,5.64,overcast clouds
10,10,Bilma,NE,18.6853,12.9164,77.67,21,69,0.89,broken clouds
17,17,Opotiki,NZ,-38.0833,177.2833,71.38,50,13,7.78,few clouds
...,...,...,...,...,...,...,...,...,...,...
681,681,Riyadh,SA,24.6877,46.7219,73.54,22,0,5.10,clear sky
682,682,Maceio,BR,-9.6658,-35.7353,78.24,88,20,4.61,few clouds
683,683,Pochutla,MX,15.7432,-96.4661,82.81,84,100,3.47,overcast clouds
688,688,Itarema,BR,-2.9248,-39.9167,79.14,81,71,13.58,broken clouds


In [7]:
# 5a. Create DataFrame called hotel_df to store hotel names along with city, country, max temp, and coordinates.
hotel_df = clean_df[["City", "Country", "Max Temp", "Current Description", "Lat", "Lng"]].copy()

# 5b. Create a new column "Hotel Name"
hotel_df["Hotel Name"] = ""

In [8]:
# 6a. Set parameters to search for hotels with 5000 meters.
params = {
    "radius": 5000,
    "type": "lodging",
    "key": g_key

}

# 6b. Iterate through the hotel DataFrame.
for index, row in hotel_df.iterrows(): 

    # 6c. Get latitude and longitude from DataFrame
    lat = row["Lat"]
    lng = row["Lng"]
    
    # Add the latitude and longitude to location key for the params dictionary.
    params["location"] = f"{lat},{lng}"
    
    # 6d. Set up the base URL for the Google Directions API to get JSON data.
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

    # 6e. Make request and retrieve the JSON data from the search. 
    hotels = requests.get(base_url, params=params).json()
    
    # 6f. Get the first hotel from the results and store the name, if a hotel isn't found skip the city.
    try:
        hotel_df.loc[index, "Hotel Name"] = hotels["results"][0]["name"]
    
    except (IndexError):
        print("Hotel not found... skipping.")   
        hotel_df.loc[index, "Hotel Name"] = "NA"
    
        print("-------------")

Hotel not found... skipping.
-------------
Hotel not found... skipping.
-------------
Hotel not found... skipping.
-------------
Hotel not found... skipping.
-------------
Hotel not found... skipping.
-------------
Hotel not found... skipping.
-------------
Hotel not found... skipping.
-------------
Hotel not found... skipping.
-------------
Hotel not found... skipping.
-------------
Hotel not found... skipping.
-------------
Hotel not found... skipping.
-------------


In [9]:
# 7. Drop the rows where there is no Hotel Name.
clean_hotel_df = hotel_df.dropna()
clean_hotel_df

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
1,Atuona,PF,78.06,few clouds,-9.8000,-139.0333,Villa Enata
3,Karratha,AU,87.64,clear sky,-20.7377,116.8463,ibis Styles Karratha
7,Kaveripattinam,IN,74.93,overcast clouds,12.4207,78.2166,PR mahal
10,Bilma,NE,77.67,broken clouds,18.6853,12.9164,Expeditions Ténére voyages
17,Opotiki,NZ,71.38,few clouds,-38.0833,177.2833,Agassiz House Opotiki Backpackers
...,...,...,...,...,...,...,...
681,Riyadh,SA,73.54,clear sky,24.6877,46.7219,"InterContinental Riyadh, an IHG Hotel"
682,Maceio,BR,78.24,few clouds,-9.6658,-35.7353,Poutur Pousada
683,Pochutla,MX,82.81,overcast clouds,15.7432,-96.4661,Hotel Posada San Jose
688,Itarema,BR,79.14,broken clouds,-2.9248,-39.9167,"Sea Lounge! Bar, Club & Restaurant"


In [14]:
# 8a. Create the output File (CSV)
output_data_file = ("../Vacation_Search/WeatherPy_vacation.csv")
# 8b. Export the City_Data into a csv
clean_hotel_df.to_csv(output_data_file, index_label="City_ID")

In [11]:
# 9. Using the template add city name, the country code, the weather description and maximum temperature for the city.
info_box_template = """
<dl>
<dt>Hotel Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# 10a. Get the data from each row and add it to the formatting template and store the data in a list.
hotel_info = [info_box_template.format(**row) for index, row in clean_hotel_df.iterrows()]

# 10b. Get the latitude and longitude from each row and store in a new DataFrame.
locations = clean_hotel_df[["Lat", "Lng"]]

In [13]:
# 11a. Add a marker layer for each city to the map. 
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)
marker_layer = gmaps.marker_layer(locations, info_box_content=hotel_info)
fig.add_layer(marker_layer)

# 11b. Display the figure
fig

Figure(layout=FigureLayout(height='420px'))